In [16]:
import pandas as pd
import numpy as np
import re, os, sys
import matplotlib.pyplot as plt

import nltk
import nltk.corpus

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/fcasas/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

<a id="cuarto"></a>
## 3. CNN sobre texto

Cuando oimos sobre redes neuronales convolucionales (CNN) normalmente pensamos en visión artificial. Las CNN fueron responsables de los principales avances en la clasificación de imágenes y son el núcleo de la mayoría de los sistemas de *Computer Vision* en la actualidad, desde el etiquetado automático de fotos de Facebook hasta los autos que conducen por sí mismos.

Más recientemente, también hemos empezado a aplicar CNN a problemas de procesamiento del lenguaje natural (NLP) y hemos obtenido resultados interesantes. Como sabemos, las redes convolucionales tienen importantes ventajas como invarianza a rotaciones y traslaciones así como la conectividad local (características de nivel inferior en una representación de nivel superior), además de lo que las hace fuertemente ventajosas, el **compartir** parámetros.


**¿Cómo se aplica esto a NLP?**  
En esta experimentación apicaremos una red CNN al dataset  __[Adzuna](https://www.kaggle.com/c/job-salary-prediction)__ que contiene cientos de miles de registros que en su mayoría corresponden a texto no estructurado versus sólo unos pocos estructurados. Los registros pueden estar en varios formatos diferentes debido a los cientos de diferentes fuentes de registros, los cuales corresponden a anuncios de empleadores en busca de trabajadores.  
Es decir, cada fila es un anuncio que, en estricto rigor, representa una sentencia típicamente trabajada como vectores de word embeddings como **word2vec** o **GloVe**. Así, para una frase de 10 palabras bajo representaciones de *embeddings* utilizando 100 dimensiones tendríamos una matriz de 10 × 100 como entrada, lo que simularía nuestra "imagen".


Su tarea es entonces, predecir el salario (valor continuo) de un determinado anuncio en base al texto indicado en éste. Igualmente puede valerse de otros atributos del anuncio como por ejemplo la ubicación, tipo de contrato, etc. 


A continuación se presenta un código de guía para leer los archivos y pre-procesarlos. Deberá añadir y realizar lo que estime conveniente.

In [4]:
stoplist = nltk.corpus.stopwords.words('english')
df = pd.read_csv("all/Train_rev1.csv")
df.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk


In [36]:
POS_TO_WORDNET = {
    'JJ':   nltk.corpus.wordnet.ADJ,
    'JJR':  nltk.corpus.wordnet.ADJ,
    'JJS':  nltk.corpus.wordnet.ADJ,
    'RB':   nltk.corpus.wordnet.ADV,
    'RBR':  nltk.corpus.wordnet.ADV,
    'RBS':  nltk.corpus.wordnet.ADV,
    'NN':   nltk.corpus.wordnet.NOUN,
    'NNP':  nltk.corpus.wordnet.NOUN,
    'NNS':  nltk.corpus.wordnet.NOUN,
    'NNPS': nltk.corpus.wordnet.NOUN,
    'VB':   nltk.corpus.wordnet.VERB,
    'VBG':  nltk.corpus.wordnet.VERB,
    'VBD':  nltk.corpus.wordnet.VERB,
    'VBN':  nltk.corpus.wordnet.VERB,
    'VBP':  nltk.corpus.wordnet.VERB,
    'VBZ':  nltk.corpus.wordnet.VERB,
}

lemmatizer = nltk.WordNetLemmatizer()

def penn_to_wn(tag):
    return POS_TO_WORDNET.get(tag,nltk.corpus.wordnet.NOUN)
    
def preproc_string(s):
    s = s.lower()
    s = re.sub(r'[^\w]', ' ',s)
    s = re.sub(r'\b[a-z]\b', ' ',  s)
    s = re.sub(r'\b[a-z][a-z]\b', ' ',  s)
    s = re.sub(r'\b[0-9]\b', ' ',  s)
    s = re.sub(r'\b[0-9][0-9]\b', ' ',  s)
    s = re.sub(r'\b[0-9][0-9][0-9]\b', ' ',  s)
    s = re.sub(r'[^\w.]', ' ', s)
    text = nltk.word_tokenize(s)
    # pos tagging
    tags = nltk.pos_tag(text)
    # lemmatization
    lemms = [lemmatizer.lemmatize(x,penn_to_wn(y)) for x,y in tags]
    # remove stopwords
    lemms = [x for x in lemms if x not in stoplist]
    return lemms

In [37]:
print(df['FullDescription'][0])
print()
print(preproc_string(df['FullDescription'][0]))

Engineering Systems Analyst Dorking Surrey Salary ****K Our client is located in Dorking, Surrey and are looking for Engineering Systems Analyst our client provides specialist software development Keywords Mathematical Modelling, Risk Analysis, System Modelling, Optimisation, MISER, PIONEEER Engineering Systems Analyst Dorking Surrey Salary ****K

['engineering', 'system', 'analyst', 'dorking', 'surrey', 'salary', 'client', 'locate', 'dorking', 'surrey', 'look', 'engineering', 'system', 'analyst', 'client', 'provide', 'specialist', 'software', 'development', 'keywords', 'mathematical', 'modelling', 'risk', 'analysis', 'system', 'model', 'optimisation', 'miser', 'pioneeer', 'engineering', 'system', 'analyst', 'dorking', 'surrey', 'salary']


In [52]:
def len_force(lemms,lim=5):
    while(len(lemms)<lim):
        lemms.append('')
    if len(lemms)>lim:
        lemms = lemms[:lim]
    return lemms

x_words = []
for index,x in df.iterrows():
    if(index%10000==0):
        print("%d/%d"%(index,df.shape[0]))
    t1 = len_force(preproc_string(str(x['Title'])),6)
    t2 = len_force(preproc_string(str(x['LocationNormalized'])),4)
    t3 = len_force(preproc_string(str(x['ContractTime'])),3)
    t4 = len_force(preproc_string(str(x['Company'])),7)
    t5 = len_force(preproc_string(str(x['FullDescription'])),30)
    final = t1 + [''] + t2 + [''] + t3 + [''] + t4 + [''] + t5
    x_words.append(final)

0/244768
10000/244768
20000/244768
30000/244768
40000/244768
50000/244768
60000/244768
70000/244768
80000/244768
90000/244768
100000/244768
110000/244768
120000/244768
130000/244768
140000/244768
150000/244768
160000/244768
170000/244768
180000/244768
190000/244768
200000/244768
210000/244768
220000/244768
230000/244768
240000/244768


In [53]:
y = df['SalaryNormalized']

### Embeddings 

En lugar de entrenar nuestros vectores embeddings utilizaremos el archivo __[Glove](https://www.kaggle.com/terenceliu4444/glove6b100dtxt#glove.6B.100d.txt)__ el cual cuenta con las representaciones vectoriales (de dimensionalidad 100) ya entrenadas sobre una amplia base de datos. Puede encontrar más detalle en https://nlp.stanford.edu/projects/glove/

In [54]:
## embedding dictionary
embedd = {}
f = open("all/glove.6B.100d.txt")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embedd[word] = coefs
f.close()

In [ ]:
# label current data
vocab_index = {}
embedd_matrix = []
x = -np.ones((len(x_words),len(x_words[0])),dtype='int')
for i in range(len(x_words)):
    if(i%10000==0):
        print("%d/%d"%(i,df.shape[0]))
    for j in range(len(x_words[0])):
        term = x_words[i][j]
        if term not in vocab_index:
            if term in embedd:
                embedd_matrix.append(embedd[term])
                vocab_index[term] = len(vocab_index)
        if term in vocab_index:
            x[i,j] = vocab_index[term]
x[x==-1] = len(vocab_index)
embedd_matrix.append(np.zeros(100))

In [ ]:
# save, just in case
np.save("all/x.npy",x)
np.save("all/e.npy",embedd_matrix)
np.save("all/y.npy",y)

0/244768
10000/244768
20000/244768
30000/244768
40000/244768
50000/244768
60000/244768
70000/244768
80000/244768
90000/244768
100000/244768
110000/244768
120000/244768
130000/244768
140000/244768
150000/244768
160000/244768
170000/244768
180000/244768
190000/244768
200000/244768
210000/244768
220000/244768
230000/244768
240000/244768


### Modelo

In [1]:
x_tr = x[:210000]
y_tr = y[:210000]
x_te = x[210000:]
y_te = y[210000:]

NameError: name 'x' is not defined

In [ ]:
def model_salary():
    max_input_length = 150
    inlayer = keras.layers.Embedding(input_dim=x.shape[1:],output_dim=100,
        weights=[embedd_matrix],input_length=max_input_lenght,trainable=False)
    front = inlayer
    #
    for n_filters in (200,300):
        for _ in range(2):
            front = keras.layers.Conv1D(n_filters,5,padding='same',
                activation='relu',kernel_initializer='he_uniform',
                                       padding="same")(front)
        front = keras.layers.MaxPooling1D(pool_size=(2,2))(front)
    #
    front = keras.layers.Flatten()(front)
    front = keras.layers.Dropout(0.5)(front)
    for k in range(3):
        front = keras.layers.Dense(550,activation='relu',
                                  kernel_initializer='he_uniform')(front)
        front = keras.layers.Dropout(0.5)(front)
    front = keras.layers.Dense(1,activation='sigmoid',
                              kernel_initializer='glorot_uniform')(front)
    miny = 0.5*np.min(y_tr)
    maxy = 1.5*np.max(y_tr)
    front = keras.layers.Lambda(lambda x: miny+x*(maxy-miny))
    #
    model = keras.models.Model(inputs=inlayer,outputs=front)
    return model

In [ ]:
model = model_salary()
model.summary()

In [ ]:
model = model_salary()

model.compile(loss='mean_absolute_error',optimizer='adam')

history = model.fit(Xtrain, y_train, validation_data=(Xval, y_val),
          epochs=25,batch_size=256)

model.save("model.h5")

In [ ]:
import json

with open('history.json', 'w') as f:
    json.dump(history.history,f)

### Evaluación de predicciones
Para las predicciones evalúe la métrica *Mean Absolute Error* (MAE)

```python
from sklearn.metrics import mean_absolute_error
print("MAE on train: ",mean_absolute_error(y_train, model.predict(Xtrain)))
print("MAE on validation: ",mean_absolute_error(y_val, model.predict(Xval)))
```

> **Intente resolver el problema experimentando con las ayudas que se entregan en el código y lo aprendido hasta ahora en el curso. Se espera que llegue a un MAE menor a 7000 en el conjunto de pruebas. No olvide documentar todo lo experimentando en este Informe Jupyter así como el argumento de sus decisiones.**